# 성능튜닝

## 1.환경준비

### (1) import

In [1]:
#라이브러리들을 불러오자.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import *

import warnings  # 경고메시지 제외

warnings.filterwarnings(action='ignore')

/Users/cyeongy/opt/anaconda3/envs/kt-aivle/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### (2) 데이터 준비

* 변수설명
    * COLLEGE : 대학 졸업여부
    * INCOME : 연수입
    * OVERAGE : 월평균 초과사용 시간(분)
    * LEFTOVER : 월평균 잔여시간비율(%)
    * HOUSE : 집값
    * HANDSET_PRICE : 스마트폰 가격
    * OVER_15MINS_CALLS_PER_MONTH : 월평균 장기통화(15분이상) 횟수
    * AVERAGE_CALL_DURATION : 평균 통화 시간
    * REPORTED_SATISFACTION : 만족도 설문조사 결과
    * REPORTED_USAGE_LEVEL : 사용도 자가진단 결과
    * CONSIDERING_CHANGE_OF_PLAN : 향후 변경계획 설문조사 결과
    * CHURN : 이탈(번호이동) 여부 (1-이탈, 0-잔류, Target 변수)


In [2]:
# 데이터를 불러옵시다.
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data = data.sample(5000, random_state=2022)

# sklearn에서는 'CHURN'을 가변수화 할 필요가 없다
# statsmodels, tensorflow(keras)에서는 가변수화가 필요하다.
# 전진선택법 함수에서 statsmodels.sm.Logit()을 사용했기 떄문에 가변수화를 진행
data['CHURN'] = data['CHURN'].map({'LEAVE': 1, 'STAY': 0})
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
3178,3179,0,119512,51,31,248566,229,5,2,very_sat,very_high,considering,1
14926,14927,1,142144,192,15,774317,581,29,4,unsat,very_little,never_thought,1
15116,15117,1,142308,0,79,306426,497,1,1,sat,little,considering,0
12733,12734,1,113385,0,0,333599,819,1,6,very_unsat,very_high,considering,1
14032,14033,1,90348,209,10,637286,360,26,4,unsat,little,actively_looking_into_it,0


## 2.데이터 준비

### (1) 데이터 정리

In [3]:
drop_cols = ['id']
data.drop(drop_cols, axis=1, inplace=True)

### (2) 데이터분할1 : x, y 나누기

In [4]:
target = 'CHURN'
x = data.drop(target, axis=1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [5]:
dumm_cols = ['REPORTED_SATISFACTION', 'REPORTED_USAGE_LEVEL', 'CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True)

### (5) 데이터분할2 : train : validation 나누기

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.3, random_state=20)

### (6) Scaling

In [7]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.선형모델 튜닝

Logistic Regression : 전진선택법
* 변수를 하나씩 늘려가면서
* AIC를 가장 낮추는 모델 찾기

### (1) 전진선택을 수행할 함수 만들기( **로지스틱 회귀** 용)

In [8]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm


def forward_stepwise_logistic(x_train, y_train):
    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({'step': [], 'feature': [], 'aic': []})

    # 
    for s in range(0, len(features)):
        result = {'step': [], 'feature': [], 'aic': []}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features:
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s + 1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)

        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop=True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min():
            break
        step_df = pd.concat([step_df, temp], axis=0).reset_index(drop=True)

        # 선택된 변수 제거
        v = temp.loc[0, 'feature'][s]
        features.remove(v)

        selected.append(v)

    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

### (2) 전진선택법 수행

In [9]:
vars, result = forward_stepwise_logistic(x_train, y_train)

Optimization terminated successfully.
         Current function value: 0.693075
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693062
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.683528
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684909
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.693042
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.686899
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692565
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692806
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693137
  

* 선택된 변수

In [10]:
vars

['OVERAGE',
 'HOUSE',
 'HANDSET_PRICE',
 'LEFTOVER',
 'REPORTED_SATISFACTION_very_sat',
 'INCOME',
 'REPORTED_SATISFACTION_sat']

In [11]:
result

,step,feature,aic
0,1.0,[OVERAGE],4786.699456
1,1.0,[HOUSE],4796.363859
2,1.0,[OVER_15MINS_CALLS_PER_MONTH],4810.294604
3,1.0,[REPORTED_SATISFACTION_very_sat],4845.064834
4,1.0,[AVERAGE_CALL_DURATION],4849.951663
...,...,...,...
114,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.309949
115,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.315315
116,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.316816
117,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.448187


In [12]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### (3) 모델링

* 전체 변수 

In [13]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

print(accuracy_score(y_val, p1))
print(classification_report(y_val, p1))

0.6333333333333333
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.65      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



* 전진선택법 변수

In [14]:
m2 = LogisticRegression()
m2.fit(x_train[vars], y_train)
p2 = m2.predict(x_val[vars])

print(accuracy_score(y_val, p2))
print(classification_report(y_val, p2))

0.634
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.66      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.64      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



## 4.하이퍼파라미터 튜닝

### (1) 필요한 함수 불러오기 

In [15]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### (2) Random Search

① 값의 범위를 지정한다.  
② 모델 선언(시도 횟수 지정)  
③ 모델링(값의 범위 내에서 시도 횟수만큼 랜덤하게 선택해서 시도한다.)  
④ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [16]:
# dictionary형태로 선언
params = {'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}
params

{'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [17]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                              , params  # hyperparameter 범위 지정.
                              , cv=5  # k-fold Cross Validation
                              , n_iter=5  # Random하게 시도할 횟수
                              )

#### ③ 모델링

In [18]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [19]:
# 튜닝 결과
model_rs.cv_results_

{'mean_fit_time': array([0.00077181, 0.00041528, 0.00069017, 0.00046916, 0.00056648]),
 'std_fit_time': array([7.45577027e-04, 5.24391679e-05, 2.94397593e-04, 2.31486638e-05,
        1.70200209e-04]),
 'mean_score_time': array([0.0466146 , 0.02658696, 0.03275561, 0.02083049, 0.01533813]),
 'std_score_time': array([0.02117038, 0.01969321, 0.01504738, 0.00117126, 0.00036243]),
 'param_n_neighbors': masked_array(data=[5, 3, 9, 48, 14],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['manhattan', 'manhattan', 'manhattan', 'euclidean',
                    'euclidean'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 5, 'metric': 'manhattan'},
  {'n_neighbors': 3, 'metric': 'manhattan'},
  {'n_neighbors': 9, 'metric': 'manhattan'},
  {'n_neighbors': 48, 'metric': 'euclidean'},
  {'n_neighbors': 14, 'metric': 'e

In [20]:
print(model_rs.cv_results_)

{'mean_fit_time': array([0.00077181, 0.00041528, 0.00069017, 0.00046916, 0.00056648]), 'std_fit_time': array([7.45577027e-04, 5.24391679e-05, 2.94397593e-04, 2.31486638e-05,
       1.70200209e-04]), 'mean_score_time': array([0.0466146 , 0.02658696, 0.03275561, 0.02083049, 0.01533813]), 'std_score_time': array([0.02117038, 0.01969321, 0.01504738, 0.00117126, 0.00036243]), 'param_n_neighbors': masked_array(data=[5, 3, 9, 48, 14],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_metric': masked_array(data=['manhattan', 'manhattan', 'manhattan', 'euclidean',
                   'euclidean'],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 5, 'metric': 'manhattan'}, {'n_neighbors': 3, 'metric': 'manhattan'}, {'n_neighbors': 9, 'metric': 'manhattan'}, {'n_neighbors': 48, 'metric': 'euclidean'}, {'n_neighbors': 14, 'metric': 'euclidean'}], 'split0_t

In [21]:
model_rs.cv_results_['params']

[{'n_neighbors': 5, 'metric': 'manhattan'},
 {'n_neighbors': 3, 'metric': 'manhattan'},
 {'n_neighbors': 9, 'metric': 'manhattan'},
 {'n_neighbors': 48, 'metric': 'euclidean'},
 {'n_neighbors': 14, 'metric': 'euclidean'}]

In [22]:
model_rs.cv_results_['mean_test_score']

array([0.59142857, 0.57314286, 0.59142857, 0.59428571, 0.57257143])

In [23]:
# 최적의 파라미터
model_rs.best_params_

{'n_neighbors': 48, 'metric': 'euclidean'}

In [24]:
# 그때의 성능
model_rs.best_score_

0.5942857142857143

In [25]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.57      0.75      0.65       738
           1       0.65      0.45      0.53       762

    accuracy                           0.60      1500
   macro avg       0.61      0.60      0.59      1500
weighted avg       0.61      0.60      0.59      1500



### (3) 실습 : Random Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [26]:
model = DecisionTreeClassifier()

params = {'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101, 10)}

#### ② 모델 선언

In [27]:
model_rs = RandomizedSearchCV(estimator=model, param_distributions=params)

#### ③ 모델링

In [28]:
model_rs.fit(x_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(),
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101, 10)})

In [29]:
model_rs.best_params_

{'min_samples_leaf': 40, 'max_depth': 8}

In [30]:
model_rs.best_score_

0.694

In [31]:
pred = model_rs.predict(x_val)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.68      0.69      0.69       738
           1       0.70      0.69      0.69       762

    accuracy                           0.69      1500
   macro avg       0.69      0.69      0.69      1500
weighted avg       0.69      0.69      0.69      1500



### (4) Grid Search

① 값의 범위를 지정한다.  
② 모델링(값의 범위 내에서 모든 조합을 다 시도한다.)  
③ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [32]:
# dictionary형태로 선언
params = {'n_neighbors': range(3, 31, 2), 'metric': ['euclidean', 'manhattan']}
params

{'n_neighbors': range(3, 31, 2), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [33]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv=5)

#### ③ 모델링

In [34]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(3, 31, 2)})

In [35]:
# 튜닝 결과
model_gs.cv_results_

{'mean_fit_time': array([0.00043397, 0.00040808, 0.0004868 , 0.00047164, 0.00043921,
        0.00049419, 0.00049596, 0.00044079, 0.0004921 , 0.00056024,
        0.00038538, 0.00050325, 0.00037942, 0.00039077, 0.00044384,
        0.00040112, 0.00036945, 0.00042238, 0.0004674 , 0.00054674,
        0.00042014, 0.00043287, 0.00048199, 0.00045004, 0.0004508 ,
        0.00049262, 0.00040026, 0.00046039]),
 'std_fit_time': array([2.01197030e-05, 9.07946032e-06, 1.24173322e-04, 5.82513531e-05,
        1.90886199e-05, 1.03722155e-04, 1.93944884e-04, 3.12397127e-05,
        7.59905505e-05, 1.46816403e-04, 1.49209654e-05, 2.09973357e-04,
        6.01004011e-06, 1.18309439e-05, 2.15652735e-05, 2.61777086e-05,
        1.65851614e-05, 3.13375262e-05, 1.51650710e-05, 1.05097268e-04,
        1.83259383e-05, 3.97049907e-05, 2.73388670e-05, 1.35924521e-05,
        1.97294219e-05, 1.11947453e-04, 2.31004843e-05, 4.50766249e-05]),
 'mean_score_time': array([0.01125765, 0.01195235, 0.01242838, 0.01290641, 

In [36]:
model_gs.cv_results_['params']

[{'metric': 'euclidean', 'n_neighbors': 3},
 {'metric': 'euclidean', 'n_neighbors': 5},
 {'metric': 'euclidean', 'n_neighbors': 7},
 {'metric': 'euclidean', 'n_neighbors': 9},
 {'metric': 'euclidean', 'n_neighbors': 11},
 {'metric': 'euclidean', 'n_neighbors': 13},
 {'metric': 'euclidean', 'n_neighbors': 15},
 {'metric': 'euclidean', 'n_neighbors': 17},
 {'metric': 'euclidean', 'n_neighbors': 19},
 {'metric': 'euclidean', 'n_neighbors': 21},
 {'metric': 'euclidean', 'n_neighbors': 23},
 {'metric': 'euclidean', 'n_neighbors': 25},
 {'metric': 'euclidean', 'n_neighbors': 27},
 {'metric': 'euclidean', 'n_neighbors': 29},
 {'metric': 'manhattan', 'n_neighbors': 3},
 {'metric': 'manhattan', 'n_neighbors': 5},
 {'metric': 'manhattan', 'n_neighbors': 7},
 {'metric': 'manhattan', 'n_neighbors': 9},
 {'metric': 'manhattan', 'n_neighbors': 11},
 {'metric': 'manhattan', 'n_neighbors': 13},
 {'metric': 'manhattan', 'n_neighbors': 15},
 {'metric': 'manhattan', 'n_neighbors': 17},
 {'metric': 'manha

In [37]:
model_gs.cv_results_['mean_test_score']

array([0.57885714, 0.57942857, 0.57714286, 0.57342857, 0.568     ,
       0.57457143, 0.58085714, 0.58257143, 0.57914286, 0.58057143,
       0.586     , 0.58457143, 0.57971429, 0.58028571, 0.57314286,
       0.59142857, 0.59171429, 0.59142857, 0.59828571, 0.59771429,
       0.59942857, 0.60142857, 0.60171429, 0.61057143, 0.61      ,
       0.61142857, 0.61857143, 0.618     ])

In [38]:
# 최적의 파라미터
model_gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 27}

In [39]:
# 그때의 성능
model_gs.best_score_

0.6185714285714285

In [40]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65       738
           1       0.65      0.51      0.57       762

    accuracy                           0.61      1500
   macro avg       0.62      0.61      0.61      1500
weighted avg       0.62      0.61      0.61      1500



### (5) 실습 : Grid Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [41]:
params = {'max_depth': range(1, 11),
          'min_samples_leaf': range(10, 101, 10)}

#### ② 모델 선언

In [45]:
practice_gs = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=params, cv=5)

#### ③ 모델링

In [46]:
practice_gs.fit(x_train_s, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(10, 101, 10)})

In [49]:
pred = practice_gs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.66      0.70      0.68       738
           1       0.69      0.66      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500



In [51]:
# 튜닝 결과
practice_gs.cv_results_

{'mean_fit_time': array([0.007231  , 0.00225768, 0.00215907, 0.00262957, 0.00176463,
        0.0021492 , 0.001999  , 0.00184603, 0.00156384, 0.00156546,
        0.00259619, 0.00257649, 0.00269861, 0.00251861, 0.00266523,
        0.00254335, 0.00245595, 0.00249391, 0.00263419, 0.00263004,
        0.00351763, 0.00353341, 0.00352716, 0.00350637, 0.00363345,
        0.00348792, 0.00348396, 0.00348282, 0.00346575, 0.00353141,
        0.00441542, 0.00438366, 0.00435491, 0.00435567, 0.0043272 ,
        0.00433612, 0.00428829, 0.00427518, 0.00437722, 0.00419745,
        0.0053504 , 0.00530057, 0.00522771, 0.00519819, 0.00510845,
        0.00504603, 0.00499644, 0.00489602, 0.00477405, 0.00462756,
        0.00621295, 0.00610118, 0.0059464 , 0.00585799, 0.00582199,
        0.00555072, 0.00538731, 0.00542469, 0.00519257, 0.0050385 ,
        0.00708156, 0.00686555, 0.00662618, 0.0064158 , 0.00612535,
        0.00578809, 0.00551581, 0.00528808, 0.00537033, 0.00481963,
        0.00767245, 0.00730829,

In [52]:
practice_gs.cv_results_['params']

[{'max_depth': 1, 'min_samples_leaf': 10},
 {'max_depth': 1, 'min_samples_leaf': 20},
 {'max_depth': 1, 'min_samples_leaf': 30},
 {'max_depth': 1, 'min_samples_leaf': 40},
 {'max_depth': 1, 'min_samples_leaf': 50},
 {'max_depth': 1, 'min_samples_leaf': 60},
 {'max_depth': 1, 'min_samples_leaf': 70},
 {'max_depth': 1, 'min_samples_leaf': 80},
 {'max_depth': 1, 'min_samples_leaf': 90},
 {'max_depth': 1, 'min_samples_leaf': 100},
 {'max_depth': 2, 'min_samples_leaf': 10},
 {'max_depth': 2, 'min_samples_leaf': 20},
 {'max_depth': 2, 'min_samples_leaf': 30},
 {'max_depth': 2, 'min_samples_leaf': 40},
 {'max_depth': 2, 'min_samples_leaf': 50},
 {'max_depth': 2, 'min_samples_leaf': 60},
 {'max_depth': 2, 'min_samples_leaf': 70},
 {'max_depth': 2, 'min_samples_leaf': 80},
 {'max_depth': 2, 'min_samples_leaf': 90},
 {'max_depth': 2, 'min_samples_leaf': 100},
 {'max_depth': 3, 'min_samples_leaf': 10},
 {'max_depth': 3, 'min_samples_leaf': 20},
 {'max_depth': 3, 'min_samples_leaf': 30},
 {'max_de

In [53]:
practice_gs.cv_results_['mean_test_score']

array([0.614     , 0.614     , 0.614     , 0.614     , 0.614     ,
       0.614     , 0.614     , 0.614     , 0.614     , 0.614     ,
       0.66142857, 0.66142857, 0.66142857, 0.66142857, 0.66142857,
       0.66142857, 0.66142857, 0.66142857, 0.66142857, 0.66142857,
       0.692     , 0.69314286, 0.69314286, 0.69314286, 0.69314286,
       0.69314286, 0.69314286, 0.69314286, 0.69314286, 0.69514286,
       0.68942857, 0.68857143, 0.68942857, 0.68971429, 0.68942857,
       0.68914286, 0.69285714, 0.69342857, 0.69742857, 0.69314286,
       0.686     , 0.69285714, 0.69514286, 0.69314286, 0.68857143,
       0.688     , 0.69057143, 0.69085714, 0.69771429, 0.69342857,
       0.67885714, 0.684     , 0.68714286, 0.68657143, 0.69114286,
       0.68742857, 0.69142857, 0.68971429, 0.69571429, 0.69228571,
       0.67371429, 0.68828571, 0.70057143, 0.69657143, 0.69      ,
       0.686     , 0.69      , 0.69114286, 0.69542857, 0.69228571,
       0.66657143, 0.688     , 0.694     , 0.69428571, 0.69257

In [54]:
# 최적의 파라미터
practice_gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 30}

In [55]:
# 그때의 성능
practice_gs.best_score_

0.7005714285714285